In [0]:
%pip install openai

In [0]:
dbutils.library.restartPython()


In [0]:
dbutils.secrets.listScopes()
dbutils.secrets.list("openai_scope")

In [0]:
%pip install -U mlflow
dbutils.library.restartPython()

In [0]:
import mlflow
mlflow.openai.autolog()

In [0]:
from openai import OpenAI
import os
from pyspark.sql import SparkSession

# Initialize OpenAI client using Databricks secret for API key
openai_api_key = dbutils.secrets.get(scope="openai_scope", key="openai_api_key")
os.environ["OPENAI_API_KEY"] = openai_api_key

client = OpenAI()

spark = SparkSession.builder.getOrCreate()

def query_openai(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant with expertise in vehicle sales and fleet data."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2,
        max_tokens=100
    )
    return response.choices[0].message.content

def get_sales_summary():
    df = spark.table("enterprise_modernization.gold.customer_vehicle_fleet")
    total_sales = df.selectExpr("sum(sales) as total_sales").collect()[0]["total_sales"]
    return f"The total sales in the vehicle fleet is {total_sales:.2f}."

def chatbot_response(user_query):
    data_summary = get_sales_summary()
    print(data_summary)
    prompt = (
        f"Data Summary: {data_summary}\n\n"
        f"User Query: {user_query}\n\n"
        "Answer the query based on the above data summary and your general knowledge."
    )
    answer = query_openai(prompt)
    return answer

# Example usage:
user_input = "What were the total sales last month and any trends you notice?"
response = chatbot_response(user_input)
print("Chatbot response:\n", response)


In [0]:
%pip install anthropic
dbutils.library.restartPython()

In [0]:
import os
from pyspark.sql import SparkSession
from anthropic import Anthropic

# Initialize Anthropic API key from Databricks secret
anthropic_api_key = dbutils.secrets.get(scope="anthropic_scope", key="anthropic_api_key")
os.environ["ANTHROPIC_API_KEY"] = anthropic_api_key

client = Anthropic(api_key=anthropic_api_key)

spark = SparkSession.builder.getOrCreate()

def query_anthropic(prompt):
    response = client.messages.create(
        model="claude-sonnet-4-5-20250929",
        messages=[
            {"role": "system", "content": "You are a helpful assistant with expertise in vehicle sales and fleet data."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=100,
    )
    return response.content

def get_sales_summary():
    df = spark.table("enterprise_modernization.gold.customer_vehicle_fleet")
    total_sales = df.selectExpr("sum(sales) as total_sales").collect()[0]["total_sales"]
    return f"The total sales in the vehicle fleet is {total_sales:.2f}."

def chatbot_response(user_query):
    data_summary = get_sales_summary()
    prompt = (
        f"Data Summary: {data_summary}\n\n"
        f"User Query: {user_query}\n\n"
        "Answer the query based on the above data summary and your general knowledge."
    )
    answer = query_anthropic(prompt)
    return answer

# Example usage:
user_input = "What were the total sales last month and any trends you notice?"
response = chatbot_response(user_input)
print("Chatbot response:\n", response)
